In [13]:
import pandas as pd
import multiprocessing
import parmap
import datetime
import os
from pathonoia import pathonoia

In [14]:
# this folder will be created and for every dataset and sample corresponding subfolders
OUTPUT_DIR = "./OUTPUT/"

# The Inputfolder should contain one or more fasta files
# These files should be samples of the same dataset
# if the dataset was originally host RNAseq, align it first to the host's transcriptome and extract the unmapped reads
# the fasta files should contain all (host-)unmapped reads 
# the unmapped reads can be extracted from the alignment BAM file with SAMtools:

# samtools view -hb -f 4 host_align.bam > host_unmapped.bam
# samtools view host_unmapped.bam | awk \'{OFS="\\t"; print ">"$1"\\n"$10}\' - > host_unmapped.fa

inputfolder = "./dataset/"

# make sure Kraken 2 is installed on your system along with its index build with k=31 AND l=31 
# Instructions for creating this index can be found at the end of this notebook
krakenTOOL="../tools/kraken2/kraken2"
krakenDB="../tools/kraken2/db/kraken2_k31"
# this database is downloadable from oasis.ims.bio/kraken2_k31.zip or oasis.dzne.de/kraken2_31.zip 
pathonoia.checkKrakenInstall(krakenTOOL,krakenDB)

True

In [15]:
# run this cell, these are definitions / functions, only

def processDataset(inputfolder):
    print(inputfolder)
    ds = inputfolder.rsplit("/", 2)[1]
    print(ds)
    argumentList = []
    for file in os.listdir(inputfolder):
        if os.path.isfile(inputfolder+file):
            if file.endswith(".fa") or file.endswith(".fasta"):           
                sample = file.replace(".fasta","").replace(".fa","")
                odir = OUTPUT_DIR + ds + "/" + sample
                s1 = inputfolder + file
                tup = (ds, odir, sample, s1)
                argumentList.append(tup)
            else:
                print(file + " is not a fasta file, PASS")
        else:
            print(file + " is not a file! PASS")
            
    print(str(len(argumentList)) + " samples to process")
    
    if not pathonoia.checkKrakenInstall(krakenTOOL, krakenDB):
        print("Problem with Kraken Installation. ABORT")
        return()

    allNucPerTaxList = parmap.starmap(runOneSample, argumentList, pm_processes=1, pm_pbar=True)
    
    print("merge and save results")
    allNucPerTax = dict()
    for i in range(len(argumentList)):
        allNucPerTax[argumentList[i][2]] = allNucPerTaxList[i]

    dsdf = pathonoia.mergeNucTaxDataframes(allNucPerTax)
    dsdf.to_csv(OUTPUT_DIR + ds + "/" + ds + "_contamination.csv")
    print(ds + " done")
    return(dsdf)

def runOneSample(ds, outputdir, samplename, sample1):   
    print(outputdir)
    if(os.path.isdir(outputdir)):
        print(outputdir + " ALREADY EXISTS")
    else:
        os.makedirs(outputdir)
    outdir_prefix = outputdir + "/" + samplename
    logfile = open(outdir_prefix + "_pathonoiaLOG_"+datetime.datetime.now().strftime("%Y%m%d")+".log", "a") 
    
    print("++++++++++++++", file = logfile)
    print(samplename + "\n", file = logfile)
    
    print("Kraken2", file = logfile)
    if (os.path.isfile(outdir_prefix + "_krakenalign.txt")):
        print("Kraken results already available, skip", file = logfile)
    else:
        execute(KRAKENcommand(outdir_prefix, sample1),logfile)

    print("evaluate", file = logfile)
    nucCutOff = 100
    print("nucCutOff = " + str(nucCutOff), file = logfile)
    nucPerTax = pathonoia.evalKrakenAlign(outdir_prefix, sample1, nucCutOff, logfile)
    print("DONE", file = logfile)
    logfile.close()
    print(samplename + " done")
    return(nucPerTax)

def execute(cmd, logfile):
    print(cmd, file = logfile)
    answer = "cmd didnt run, uncommented"
    answer = os.system(cmd)
    print(answer, file = logfile)    

def KRAKENcommand(outdir_prefix, sample):
    #if you want to change settings here, make sure the spaces in the string stay the way they are
    krakenPARAMETERS = "--db " + krakenDB + " --use-names --threads 8 --memory-mapping "
    outputPARAMETER = "--output " + outdir_prefix + "_krakenalign.txt --report " + outdir_prefix + "_krakenreport.txt "
    samplePARAMETER = sample
    cmd = krakenTOOL + " " + krakenPARAMETERS + outputPARAMETER + samplePARAMETER
    return(cmd)

In [17]:
# This will take time depending on dataset size
# The terminal in which the notebook is running may give additional output

ds_contamination = processDataset(inputfolder)

./dataset/
dataset
.ipynb_checkpoints is not a file! PASS
5 samples to process


  0%|          | 0/5 [00:00<?, ?it/s]

./OUTPUT/dataset/buccal_small
buccal_small done
./OUTPUT/dataset/cityparks_small
cityparks_small done
./OUTPUT/dataset/hous1_small


 40%|████      | 2/5 [02:04<03:06, 62.15s/it]

hous1_small done
./OUTPUT/dataset/hous2_small
hous2_small done
./OUTPUT/dataset/soil_small


 80%|████████  | 4/5 [04:22<01:06, 66.29s/it]

soil_small done


6it [06:15, 62.57s/it]                       

merge and save results
dataset done


In [18]:
ds_contamination

,species_name,phylo_level,parent,buccal_small,cityparks_small,hous1_small,hous2_small,soil_small
1189310,Myxococcus macrosporus DSM 14697,S1,35,NaN,1636235.0,NaN,NaN,1632272.0
35,Myxococcus macrosporus,S,32,NaN,1632307.0,NaN,NaN,1629304.0
208963,Pseudomonas aeruginosa UCBPP-PA14,S1,287,NaN,NaN,1534983.0,NaN,NaN
208964,Pseudomonas aeruginosa PAO1,S1,287,NaN,NaN,1527298.0,NaN,NaN
287,Pseudomonas aeruginosa,S,136841,NaN,NaN,1526802.0,NaN,NaN
509170,Acinetobacter baumannii SDF,S1,470,NaN,NaN,NaN,1007666.0,NaN
1096997,Acinetobacter baumannii BJAB0868,S1,470,NaN,NaN,NaN,1007417.0,NaN
470,Acinetobacter baumannii,S,909768,NaN,NaN,NaN,1007309.0,NaN
1785128,Acinetobacter lactucae,S,909768,NaN,NaN,NaN,888271.0,NaN
374930,Haemophilus influenzae PittEE,S1,727,840191.0,NaN,NaN,NaN,NaN


In [ ]:
# continue processing the dataset with downstream_analysis notebook

# setup Kraken 2 for Pathonoia
follow these commands one by one to build a Kraken Index that works with Pathonoia

    git clone https://github.com/DerrickWood/kraken2.git
    cd kraken2/
    ./install_kraken2.sh .
    mkdir db
    ./kraken2-build --download-taxonomy --db db/bacvir_k31 #?--skip-maps
    ./kraken2-build --download-library bacteria --db db/bacvir_k31 --use-ftp
    ./kraken2-build --download-library viral --db db/bacvir_k31 --use-ftp

    ./kraken2-build --build --db db/bacvir_k31 --threads 8 --kmer-len 31 --minimizer-len 31

    ./kraken2-build --clean --db db/bacvir_k31